```{=latex}
\usepackage{hyperref}
\usepackage{graphicx}
\usepackage{listings}
\usepackage{textcomp}
\usepackage{fancyvrb}

\newcommand{\passthrough}[1]{\lstset{mathescape=false}#1\lstset{mathescape=true}}
\newcommand{\tightlist}{}
```

```{=latex}
\title{End-to-End Kubernetes}
\author{Moshe Zadka -- https://cobordism.com}
\date{}

\begin{document}
\begin{titlepage}
\maketitle
\end{titlepage}

\frame{\titlepage}
```

```{=latex}
\begin{frame}
\frametitle{Acknowledgement of Country}

Belmont (in San Francisco Bay Area Peninsula)

Ancestral homeland of the Ramaytush Ohlone

\end{frame}
```

## Introduction

### Examples as caricatures

```{=latex}
\begin{frame}
\frametitle{Examples as Caricatures}

Realistic exaggeration

\end{frame}
```

### Example application

```{=latex}
\begin{frame}
\frametitle{A Caricature}
```

In [ ]:
# e2e_k8s/demo.py
from pyramid import response, config, view
@view.view_config(route_name='add')
def add(request):
    x, y = (int(request.matchdict[c]) for c in "xy")
    return response.Response(str(x+y))
with config.Configurator() as _cfg:
    _cfg.add_route("add", "/add/{x}/{y}")
    _cfg.scan("e2e_k8s")
    application = _cfg.make_wsgi_app()


```{=latex}
\end{frame}
```

### Software as a pipeline

```{=latex}
\begin{frame}
\frametitle{Software as a Pipeline}

Code $\rightarrow$ Customer

\end{frame}
```

### Software as a leaky pipeline

```{=latex}
\begin{frame}
\frametitle{Leaky Pipeline}

Code\pause

Unit tests\pause

Running locally\pause

"Simulation" environments\pause

Production\pause

Customer

\end{frame}
```

```{=latex}
\begin{frame}
\frametitle{Leaky Pipeline}

Leak:\pause

Problem happening at one stage but not the previous one
\end{frame}
```

### Goal: fixing leaks

```{=latex}
\begin{frame}
\frametitle{Fixing Leaks}

Less differences -- less leaks
\end{frame}
```

### Talk goal: Mental model

```{=latex}
\begin{frame}
\frametitle{Model, Not Solutions}

Hard problem to solve\pause

Better mental model for local solutions

\end{frame}
```

## Kubernetes Crash Course

### What is kubernetes?

```{=latex}
\begin{frame}
\frametitle{Kubernetes}

Orchestration: Running containers on compute resources

\end{frame}
```

### Why is kubernetes popular?

```{=latex}
\begin{frame}
\frametitle{Why Kubernetes?}

\begin{itemize}
\item Builds on containers
\item Documented REST API
\item Extensible
\end{itemize}

\end{frame}
```

### What is DevOps?

```{=latex}
\begin{frame}
\frametitle{What is DevOps (Not?}}

an engineering speciality\pause

"developers should run their code"\pause

"no need for operations"\pause

"developers working for ops"\pause

"ops working for developers"\pause

\end{frame}

\begin{frame}
\frametitle{What is DevOps?}

Developers, ops (and QA, and security, ....)
collaborating on a pipeline

\end{frame}
```

### Why is DevOps good?

```{=latex}
\begin{frame}
\frametitle{How and WhyDevOps}

Reduce friction\pause

Not just technical solution\pause

Part of technical part: reduce differences between
developer environment
and
production environment

\end{frame}
```

## Scope

### Multi-cluster (beyond scope)

### Database migration (beyond scope)

### Stateful applications (beyond scope)

### Advanced deployment (beyond scopre)

## Software Development Lifecycle

### What is an edit debug cycle?

### Why edit-debug cycle latency matters

## Kubernetes Application Architecture

### Containers

```{=latex}
\begin{frame}
\frametitle{Containers}

"Light weight VMs"\pause

"Heavy weight processes"

\end{frame}
```

### Pods

```{=latex}
\begin{frame}
\frametitle{Pods}

Groups of containers\pause

Share network namespace\pause

Can share process namespace\pause

\end{frame}
```

### Deployments

```{=latex}
\begin{frame}
\frametitle{Deployment}

Routable set of identity-less pods

\end{frame}
```

### Stateful Ses

```{=latex}
\begin{frame}
\frametitle{StatefulSet}

Set of identifiable pods


\end{frame}
```

## Environments

### What is an environment?

```{=latex}
\begin{frame}
\frametitle{Environment}

A collection of services which work together

\end{frame}
```

### Cross-talk

```{=latex}
\begin{frame}
\frametitle{Environment Cross-Talk}

Environments (mostly) don't cross-talk

\end{frame}
```

### Regional Separation

```{=latex}
\begin{frame}
\frametitle{Regional Environments}

Jursidictional/Geographical

\end{frame}
```

### Maturity Separation

```{=latex}
\begin{frame}
\frametitle{Maturity-Based Environments}

Production\pause

Stagingin, Testing....\pause

Ad-hoc Remote\pause

Local


\end{frame}
```

## Modern SDLC Caricature

```{=latex}
\begin{frame}
\frametitle{Software Development Lifecycle (Caricature)}

Develop\pause

Review\pause

Merge\pause

Deploy

\end{frame}
```

### Develop

```{=latex}
\begin{frame}
\frametitle{SDLC: Develop}

Write code


\end{frame}
```

### Review

```{=latex}
\begin{frame}
\frametitle{SDLC: Review}

Approve/Reject


\end{frame}
```

### Merge

```{=latex}
\begin{frame}
\frametitle{SDLC: Merge}

Integrate code into rest of product

\end{frame}
```

### Deploy

```{=latex}
\begin{frame}
\frametitle{SDLC: Deploy}

Run in production

\end{frame}
```

### No "test" stage

```{=latex}
\begin{frame}
\frametitle{SDLC: No Test?}

Every stage contains testing:\pause

Develop: unit/ad-hoc/local \pause

Review: Continuous Integration testing \pause

Merge: Continuous Integration testing \pause

Deploy: Monitoring and alerting 

\end{frame}
```

## Kubernetes across environments

```{=latex}
\begin{frame}
\frametitle{K8s across Environments}

Environments are like k8s clusters \pause

Can they be the same?

\end{frame}
```

### Why

```{=latex}
\begin{frame}
\frametitle{Clusters as Environments: Why}

Production will be k8s probably\pause

Less leaks!
\end{frame}
```

### Challenges

```{=latex}
\begin{frame}
\frametitle{Clusters as Environments: How}

Different sizes \pause

Different platforms\pause

Different versions

\end{frame}
```

### Lima

```{=latex}
\begin{frame}
\frametitle{Local environments: Lima}

Run a VM running containers on Mac

\end{frame}
```

### WSL2

```{=latex}
\begin{frame}
\frametitle{Local environments: Lima}

Run a VM (that can runn containers) on Windows

\end{frame}
```

### Minikube

```{=latex}
\begin{frame}
\frametitle{Local environments: Minikube}

Single-host kubernetes \pause

Lima, WSL2, Local linux, Remote VM....

\end{frame}
```

### Cloud

```{=latex}
\begin{frame}
\frametitle{Remote environments: Cloud native}

Part of cloud offerings!

\end{frame}
```

## Customizing Kubernetes

```{=latex}
\begin{frame}
\frametitle{Customizing Kubernetes}

One size\pause

literally\pause

does not fit all environments.

\end{frame}
```

### Ad-hoc

```{=latex}
\begin{frame}
\frametitle{Customizing Kubernetes: Ad-hoc}

Never underestimate a programmer with sed

\end{frame}
```

### Template

```{=latex}
\begin{frame}
\frametitle{Customizing Kubernetes: Template}

Use a generic templating language (e.g., Jinja2)

\end{frame}
```

### Kustomize

```{=latex}
\begin{frame}
\frametitle{Customizing Kubernetes: Kustomize}

Part of kubectl \pause

YAML-based YAML-editing DSL

\end{frame}
```

### Helm

```{=latex}
\begin{frame}
\frametitle{Customizing Kubernetes: Helm}

Specialized templating system

\end{frame}
```

### Operators

```{=latex}
\begin{frame}
\frametitle{Customizing Kubernetes: Operators}

Convert "abstract" description to "concrete" description

\end{frame}
```

### Server-Side Apply

```{=latex}
\begin{frame}
\frametitle{Customizing Kubernetes: Server-Side Apply}

Combine YAML from different sources

\end{frame}
```

## Kubernetes Pods

### What are pods?

```{=latex}
\begin{frame}
\frametitle{Kubernetes fundamentals: Pods}

Low-level "run several containers" \pause

Usually not directly used\pause

Almost always indirectly used
\end{frame}
```

### Container namespaces

```{=latex}
\begin{frame}
\frametitle{Kubernetes fundamentals: Pods and Namespacing}

Share network\pause

Share process-space (optional)\pause

Share temporary storage (optional)\pause

\end{frame}
```

### Super-Pods namespaces

```{=latex}
\begin{frame}
\frametitle{Kubernetes fundamentals: Things that wrap Pods}

Job\pause

CronJob\pause

ReplicaSet\pause

StatefulSet\pause

Deployment
\end{frame}
```

## Kubernetes routing

### Pod

```{=latex}
\begin{frame}
\frametitle{Kubernetes routing: Pod}

Unique IP\pause

DNS depends on name
\end{frame}
```

### Service

```{=latex}
\begin{frame}
\frametitle{Kubernetes routing: Service}

Select "participating" pods \pause

Regular: route (usually TCP) to Pods\pause

Headless: DNS to Pods 

\end{frame}
```

### StatefulSet

```{=latex}
\begin{frame}
\frametitle{Kubernetes routing: StatefulSet}

With Service\pause

Route to "name-number".suffix\pause

\end{frame}
```

## Updating Containers

```{=latex}
\begin{frame}
\frametitle{Kubernetes-based Development: Basic}

Build new container\pause

Configure k8s with new contaier\pause

Repeat

\end{frame}
```

```{=latex}
\begin{frame}
\frametitle{Kubernetes-based Development: Basic is Slow}

Rebuild container (even with cache)\pause

Redownload container\pause

Restart container

\end{frame}
```

```{=latex}
\begin{frame}
\frametitle{Kubernetes-based Development: Update Containers In-place}

Quicker\pause

less accurate\pause

feedback
\end{frame}
```

### SSH to Pod

```{=latex}
\begin{frame}
\frametitle{Kubernetes-based Development: SSH to Pod}

With customization\pause

Add container to pod\pause

Running ssh server

\end{frame}
```

### Modifying files

```{=latex}
\begin{frame}
\frametitle{Kubernetes-based Development: Cross-Pod file-access}

Share process namespace\pause

Use proc filesystem\pause

SSH pod can modify files in pod-friend

\end{frame}
```

### Synchornizing files

```{=latex}
\begin{frame}
\frametitle{Kubernetes-based Development: Sync files}

Continuous sync\pause

Over SSH\pause

Over Pod
\end{frame}
```

### Auto-restarting

```{=latex}
\begin{frame}
\frametitle{Kubernetes-based Development: Auto-restarting}

Use customization\pause

watchmedo and friends

\end{frame}
```

## Putting it all together

```{=latex}
\begin{frame}
\frametitle{Kubernetes: End-to-End}

Put together the pieces

\end{frame}
```

### Build container

```{=latex}
\begin{frame}
\frametitle{End-to-End: Build}

Build container using rootless daemonless buildkit\pause

Push to registry

\end{frame}
```

### Configure Pod

```{=latex}
\begin{frame}
\frametitle{End-to-End: Pod}

Customize with SSH and Sync-Target

\end{frame}
```

### Sync files

```{=latex}
\begin{frame}
\frametitle{End-to-End: Local}

SSH and Sync-Source

\end{frame}
```

### Update container

```{=latex}
\begin{frame}
\frametitle{End-to-End: Update}

Restart on file changes

\end{frame}
```

### See results

```{=latex}
\begin{frame}
\frametitle{End-to-End: Feedback}

Check results (manually or automatically)

\end{frame}
```

## Summary

### K8s as a universal platform

```{=latex}
\begin{frame}
\frametitle{Kubernetes: Not Only Last Stage}

It can run everywhere!

\end{frame}
```

### K8s end-to-end

```{=latex}
\begin{frame}
\frametitle{Kubernetes as a Leak Stopper}

End-to-end on same platform

\end{frame}
```

```{=latex}
\end{document}
```